### Action Space:
- Defined as a continuous space with two variables (heating and cooling setpoints).
- low and high define the bounds of these setpoints.
- The shape indicates that there are two actions the agent can take.

### Time Variables:
- The agent will receive information about the current month, day, and hour as part of its observation.

### Observation Variables:
- The agent observes the outdoor_temperature, htg_setpoint, clg_setpoint, air_temperature, air_humidity, and HVAC_electricity_demand_rate.

### Actuators:
- These define the specific schedules or controls the agent can influence, such as Heating_Setpoint_RL and Cooling_Setpoint_RL.

### Reward Function:
- Uses LinearReward, which balances thermal comfort (air_temperature) and energy consumption (HVAC_electricity_demand_rate).


In [3]:
!pip install stable-baselines3
!pip install sinergym
!pip install wandb

  Obtaining dependency information for stable-baselines3 from https://files.pythonhosted.org/packages/06/6a/c3098a78a63b5a48e18c11d80b8c532f8b7785d6abb1329cfe3034572161/stable_baselines3-2.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.13 from https://files.pythonhosted.org/packages/80/83/9b7681e41e59adb6c2b042f7e8eb716515665a6eed3dda4215c6b3385b90/torch-2.4.0-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/01/75/6c7ce560e95714a10fcbb3367d1304975a1a3e620f72af28921b796403f3/matplotlib-3.9.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/ae/f0/48285f0262fe47103a4a45972ed2f9b93e4c80b8fd609fa98da78b2a5706/filelock-3.15.4-py3-none-any.whl.metadata
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/c1/f9/6845bf8fca0eaf847d

In [14]:
import os

# Setting environment variable path for EPLUS (Ubuntu)
os.environ['EPLUS_PATH'] = '/usr/local/EnergyPlus-23-1-0' 
# Add pyenergyplus path to PYTHONPATH (Ubuntu)
pyenergyplus_path = '/usr/local/EnergyPlus-23-1-0/pyenergyplus'
os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + ':' + pyenergyplus_path
# Creating environment and applying wrappers for normalization and logging

# Defining and Registering own environment

In [7]:
# import json
# import os
# import gymnasium as gym
# from gymnasium.envs.registration import register
# from sinergym.utils.common import convert_conf_to_env_parameters
# from sinergym.utils.constants import *
# from sinergym.utils.rewards import *

# # Set the version (assumed correctly set)
# current_directory = os.getcwd()
# version_file = os.path.join(current_directory, 'sinergym', 'version.txt')
# with open(version_file, 'r') as file_handler:
#     __version__ = file_handler.read().strip()

# print(__version__)

# # Registering custom environment
# register(
#     id='Eplus-hitl-continuous-v1',
#     entry_point='sinergym.envs:EplusEnv',
#     kwargs={
#         'building_file': '1ZoneDataCenterCRAC_wApproachTemp.epJSON',
#         'weather_files': 'USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw',
#         'action_space': gym.spaces.Box(
#             low=np.array([15.0, 22.5], dtype=np.float32),
#             high=np.array([22.5, 30.0], dtype=np.float32),
#             shape=(2,),
#             dtype=np.float32),
#         'time_variables': ['month', 'day_of_month', 'hour'],
#         'variables': {
#             'outdoor_temperature': (
#                 'Site Outdoor Air DryBulb Temperature',
#                 'Environment'),
#             'htg_setpoint': (
#                 'Zone Thermostat Heating Setpoint Temperature',
#                 'SPACE5-1'),
#             'clg_setpoint': (
#                 'Zone Thermostat Cooling Setpoint Temperature',
#                 'SPACE5-1'),
#             'air_temperature': (
#                 'Zone Air Temperature',
#                 'SPACE5-1'),
#             'air_humidity': (
#                 'Zone Air Relative Humidity',
#                 'SPACE5-1'),
#             'HVAC_electricity_demand_rate': (
#                 'Facility Total HVAC Electricity Demand Rate',
#                 'Whole Building')
#         },
#         'meters': {},
#         'actuators': {
#             'Heating_Setpoint_RL': (
#                 'Schedule:Compact',
#                 'Schedule Value',
#                 'HTG-SETP-SCH'),
#             'Cooling_Setpoint_RL': (
#                 'Schedule:Compact',
#                 'Schedule Value',
#                 'CLG-SETP-SCH')
#         },
#         'reward': LinearReward,
#         'reward_kwargs': {
#             'temperature_variables': ['air_temperature'],
#             'energy_variables': ['HVAC_electricity_demand_rate'],
#             'range_comfort_winter': (20.0, 23.5),
#             'range_comfort_summer': (23.0, 26.0)},
#         'env_name': 'hitl-v1',
#         'config_params': {
#             'runperiod': (1, 1, 1991, 1, 3, 1991),
#             'timesteps_per_hour': 1
#         }})

# print("Environment 'Eplus-hitl-v1' registered successfully.")


3.4.0
Environment 'Eplus-hitl-v1' registered successfully.


# PPO Training

a242ed6824522612fda77eccee41f84ee30b4964

In [22]:
import sys
from datetime import datetime

import gymnasium as gym
import numpy as np
import wandb
from stable_baselines3 import *
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.logger import HumanOutputFormat
from stable_baselines3.common.logger import Logger as SB3Logger
from stable_baselines3.common.monitor import Monitor

import sinergym
from sinergym.utils.callbacks import *
from sinergym.utils.constants import *
from sinergym.utils.logger import CSVLogger, WandBOutputFormat
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import *

# Environment ID
environment = "Eplus-5zone-mixed-continuous-stochastic-v1"
# Training episodes
episodes = 1
#Name of the experiment
experiment_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
experiment_name = 'SB3_PPO-' + environment + \
    '-episodes-' + str(episodes)
experiment_name += '_' + experiment_date


# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_PPO'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'HITL_HVAC',
                "entity": 'adityakuma0308'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

env = gym.make(environment, env_name=experiment_name)
eval_env = gym.make(environment, env_name=experiment_name+'_EVALUATION')

env = NormalizeObservation(env)
env = NormalizeAction(env)
env = LoggerWrapper(env)

eval_env = NormalizeObservation(eval_env)
eval_env = NormalizeAction(eval_env)
eval_env = LoggerWrapper(eval_env)


model = PPO('MlpPolicy', env, verbose=1)


callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    eval_env=eval_env,
    train_env=env,
    best_model_save_path=eval_env.get_wrapper_attr('workspace_path') +
    '/best_model/',
    log_path=eval_env.get_wrapper_attr('workspace_path') +
    '/best_model/',
    eval_freq=(eval_env.get_wrapper_attr('timestep_per_episode') - 1) * 2 - 1,
    deterministic=True,
    render=False,
    n_eval_episodes=1)
callbacks.append(eval_callback)


# wandb logger and setting in SB3
logger = SB3Logger(
    folder=None,
    output_formats=[
        HumanOutputFormat(
            sys.stdout,
            max_length=120),
        WandBOutputFormat()])
model.set_logger(logger)
# Append callback
log_callback = LoggerCallback()
callbacks.append(log_callback)


callback = CallbackList(callbacks)

timesteps = episodes * (env.get_wrapper_attr('timestep_per_episode') - 1)

model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=100)

model.save(str(env.get_wrapper_attr('timestep_per_episode')) + '/' + experiment_name)

env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [SB3_PPO-Eplus-5zone-mixed-continuous-stochastic-v1-episodes-1_2024-08-14_20:02]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/home/aditya-kumar/Desktop/UROP/hitl_hvac/sinergym_environment/Eplus-env-SB3_PPO-Eplus-5zone-mixed-continuous-stochastic-v1-episodes-1_2024-08-14_20:02-res35041]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds

/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/opyplus/weather_data/weather_data.py:531: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[k].fillna(value=np.nan, inplace=True)
/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/opyplus/weather_data/weather_data.py:533: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/home/aditya-kumar/Desktop/UROP/hitl_hvac/sinergym_environment/Eplus-env-SB3_PPO-Eplus-5zone-mixed-continuous-stochastic-v1-episodes-1_2024-08-14_20:02_EVALUATION-res35041]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment SB3_PPO-Eplus-5zone-mixed-co

/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/opyplus/weather_data/weather_data.py:531: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[k].fillna(value=np.nan, inplace=True)
/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/opyplus/weather_data/weather_data.py:533: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

[MODELING] (INFO) : Weather file USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw]


...

--------------------------------------------------------------------------------------------------------------| 1%
| action_network/                 |              |
|    Cooling_Setpoint_RL          | 0.063374095  |
|    Heating_Setpoint_RL          | 0.38448673   |
| action_simulation/              |              |
|    Cooling_Setpoint_RL          | 26.838888    |
|    Heating_Setpoint_RL          | 19.787737    |
| normalized_observation/         |              |
|    HVAC_electricity_demand_rate | -0.45032725  |
|    air_humidity                 | 1.0933574    |
|    air_temperature              | -0.99444073  |
|    clg_setpoint                 | 0.52806926   |
|    co2_emission                 | 0.0          |
|    day_of_month                 | 4.898854     |
|    diffuse_solar_radiation      | -0.61689425  |
|    direct_solar_radiation       | -0.58669466  |
|    hour                         | -1.5447197   |
|    htg_setpoint                 | -0.755934    |
|    month        

/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/opyplus/weather_data/weather_data.py:531: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[k].fillna(value=np.nan, inplace=True)
/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/opyplus/weather_data/weather_data.py:533: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the i

[ENVIRONMENT] (INFO) : Saving episode output path... [/home/aditya-kumar/Desktop/UROP/hitl_hvac/sinergym_environment/Eplus-env-SB3_PPO-Eplus-5zone-mixed-continuous-stochastic-v1-episodes-1_2024-08-14_20:02-res35041/Eplus-env-sub_run2/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/home/aditya-kumar/Desktop/UROP/hitl_hvac/sinergym_environment/Eplus-env-SB3_PPO-Eplus-5zone-mixed-continuous-stochastic-v1-episodes-1_2024-08-14_20:02-res35041/Eplus-env-sub_run2/USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3_Random_1.0_0.0_0.001.epw', '-d', '/home/aditya-kumar/Desktop/UROP/hitl_hvac/sinergym_environment/Eplus-env-SB3_PPO-Eplus-5zone-mixed-continuous-stochastic-v1-episodes-1_2024-08-14_20:02-res35041/Eplus-env-sub_run2/output', '/home/aditya-kumar/Desktop/UROP/hitl_hvac/sinergym_environment/Eplus-env-SB3_PPO-Eplus-5zone-mixed-continuous-stochastic-v1-episodes-1_2024-08-14_20:02-res35041/Eplus-env-sub_run2/5ZoneAutoDXVAV.epJSON']
[ENVIRONMENT] (INFO) : Episode 2 started.
[

/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.name to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.name` for environment variables or `env.get_wrapper_attr('name')` that will search the reminding wrappers.
  logger.warn(
/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.observation_variables to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.observation_variables` for environment variables or `env.get_wrapper_attr('observation_variables')` that will search the reminding wrappers.
  logger.warn(
/home/aditya-kumar/.pyenv/versions/3.11.5/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.unwrapped_observation to get variables from other wrappers is deprecated and will be remov

-------------------------------------------------
| action_network/                 |             |
|    Cooling_Setpoint_RL          | -0.49684545 |
|    Heating_Setpoint_RL          | 1.0         |
| action_simulation/              |             |
|    Cooling_Setpoint_RL          | 24.948147   |
|    Heating_Setpoint_RL          | 23.25       |
| normalized_observation/         |             |
|    HVAC_electricity_demand_rate | -0.20827356 |
|    air_humidity                 | -1.007467   |
|    air_temperature              | 0.3016209   |
|    clg_setpoint                 | -1.2188374  |
|    co2_emission                 | 0.0         |
|    day_of_month                 | -1.6680857  |
|    diffuse_solar_radiation      | 0.22381593  |
|    direct_solar_radiation       | 0.01586927  |
|    hour                         | 0.36227074  |
|    htg_setpoint                 | -1.2546775  |
|    month                        | -1.5978811  |
|    outdoor_humidity             | -1.7667835  |


# DQN Training

# DQRN Training (DQN with Recurrent Neural Networks)